In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

from nltk import RegexpParser
from nltk import pos_tag
import re
from fuzzywuzzy import process, fuzz

from keybert import KeyBERT

# doc = "WHOLE BLACK PEPPER LASTA (IMPORT TO EOU FOR RE- EXPORT)"
model = KeyBERT('distilbert-base-nli-mean-tokens')
# keywords = model.extract_keywords(doc)



def most_similar(doc_id,similarity_matrix,matrix,doc):
    #print (f'Document: {doc[doc_id]}')
    #print ('\n')
    #print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        #print('\n')
        #print (f'Document: {doc[ix]}')
        #print (f'{matrix} : {similarity_matrix[doc_id][ix]}')
    #print (f'{similarity_matrix[doc_id][-1]}')
    
    try:
        print(similarity_matrix[doc_id])
        return sorted(similarity_matrix[doc_id], reverse = True)[1]
    except:
        return 0


def key_ex(x):
    chunks = []
    for i in range(1,5):
        ls1 = model.extract_keywords(x, keyphrase_ngram_range=(1, i))

        for j in ls1:
            chunks.append(j)
    return chunks

def sel_top(val):
    final_val = []
    num = 0
    for value in val:
        print(value)
        if num > 5:
            break

        if value[1] > 0.3:
            final_val.append(value[0])
            num = num+1
    return final_val

# query = 'any MOTOR FOR INDUSTRIAL USE'

def semantic_check(query,df):
    result = []
    for x in df['Description'].values:
    #     doc = df['Description'].values
    #     doc.append(query)
#         print(x)
    #     chunks = []

        chunks = key_ex(x)
        chunks = set(chunks)

        q_c = set(key_ex(query))

        val = sorted(chunks, key = lambda x : x[1], reverse = True )
#         print(val)



        print("\n\nFrom Specification")
        final_val = sel_top(val)
        print("\n\nFrom User Description")
        final_val_query = sel_top(q_c) 
        max_sim = []
#         print(final_val_query)
        for q in final_val_query :

            final_val.append(q)
        #     print(doc)
            print("\n\n Checking distance of {} with the rest {} one by one" .format(final_val[-1], final_val[:]))
#             print(final_val)
            document_embeddings = sbert_model.encode(final_val)

            pairwise_similarities=cosine_similarity(document_embeddings)
            pairwise_differences=euclidean_distances(document_embeddings)
            max_sim.append(most_similar(-1,pairwise_similarities,'Cosine Similarity',final_val))
    #         most_similar(-1,pairwise_differences,'Euclidean Distance', final_val)

            final_val = final_val[:-1]
        max_val = max(max_sim)
        print("\n\n max_sim ",max_val)
        if max_val >= 0.85:
             result.append('yes')
        else:
            result.append("no")
        print("\n\n")
    return result



def extract_quants(data):
    data = data.replace('\n', ' ')
    data = data.replace(';', ' ')
    data = data.replace(',', ' ')
    data = data.replace('’', ' ')
    data = data.replace('‘', ' ')
    data = data.replace('  '," ")
    data = data.replace('   '," ")
    data = data.replace('#'," ")
    data = re.sub(r'\s+'," ",data)
    data = re.sub(r'\s+$',"",data)
    data = re.sub(r'(\(|\[|\{).*(\)|\]|\})',"",data)
    return data

def reduce_data(df1):
    db=pd.DataFrame(columns=["hsn","description"])
    for i in range(len(df1)):
        l= df1.iloc[i,1]
        l= re.sub("(\(|\[|\{).*$","",l)
        l= re.sub("\d{2,}","",l)
        flag=1
        for j in range(len(db)):
            k = db.iloc[j,1]
            if(fuzz.token_set_ratio(l.lower(),k.lower())>90):
                flag=0
        if flag==1:
            db=db.append(pd.DataFrame({"hsn":df1.iloc[i,0],"description":l},index=[0]), ignore_index=True)
    return db

def key_from_description(df):
    fin=pd.DataFrame(columns=["hsn","description"])
    keys = []
    for i in range(len(df)):
        try:
            unit= extract_quants(df.iloc[i]['description'])
        except:
            unit= extract_quants(df.iloc[i]['Description'])
        pattern1 = """
        P1: {<NN|NNP|NNS|DT|CC|VBZ|JJ|IN|VBN>?<NN|NNP|NNS>}
        """
        PChunker = RegexpParser(pattern1)
        res = ""
        try:
            result=PChunker.parse(pos_tag(unit.split()))

            for node in result:
                if node.label()=="P1":
                    tmp=""
                    for leaf in node.leaves():
                        tmp+=leaf[0]+" "
                    print(tmp)
                    res= res + tmp + ", "


    #             else:
    #                 fin=fin.append(pd.DataFrame({"hsn":df.iloc[i,1],"description":"NULL"},index=[0]), ignore_index=True)
        except:
            pass
    #         res.append("NULL")
        print("Res : ", res)
    #         fin=fin.append(pd.DataFrame({"hsn":df.iloc[i,1],"description":"NULL"},index=[0]), ignore_index=True)
        keys.append(res)
    try:
        fin['hsn'] = df['hsn'].values
    except:
        fin['hsn'] = df['HSN_Code'].values

    fin['description'] = keys
    #     fin=fin.append(pd.DataFrame({"hsn":df.iloc[i,1],"description":res},index=[0]), ignore_index=True)
    #         
            #fin=fin.append(pd.DataFrame({"hsn":df.iloc[i,0],"description":""},index=[0]), ignore_index=True)
    # fin= reduce_data(fin)
    print(fin)
    return fin

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel("Test file-1.xlsx")
query = "AC Motor for industrial use"
result = semantic_check(query, df)



From Specification
('ac motor', 1.0)
('ac', 0.8593)
('motor', 0.8075)


From User Description
('use', 0.4857)
('ac motor industrial use', 0.9578)
('ac motor industrial', 0.918)
('ac motor', 0.8242)
('motor', 0.6969)
('ac', 0.6493)
('motor industrial', 0.818)


 Checking distance of use with the rest ['ac motor', 'ac', 'motor', 'use'] one by one
[0.51019114 0.8280917  0.75525415 1.        ]


 Checking distance of ac motor industrial use with the rest ['ac motor', 'ac', 'motor', 'ac motor industrial use'] one by one
[0.9004084  0.5848563  0.59126663 0.9999999 ]


 Checking distance of ac motor industrial with the rest ['ac motor', 'ac', 'motor', 'ac motor industrial'] one by one
[0.9181597  0.65114915 0.6326751  0.99999994]


 Checking distance of ac motor with the rest ['ac motor', 'ac', 'motor', 'ac motor'] one by one
[1.0000002  0.75076145 0.72345567 1.0000002 ]


 Checking distance of motor with the rest ['ac motor', 'ac', 'motor', 'motor'] one by one
[0.72345567 0.746824   1.0000

[0.5316514  0.42058703 0.72845256 0.58261245 0.6741378  0.34799019
 0.99999994]


 Checking distance of ac motor with the rest ['motors output exceeding 37', 'output exceeding 37', 'motors output', 'motors output exceeding', 'motors', 'exceeding 37', 'ac motor'] one by one
[0.55523443 0.47826877 0.7679001  0.6263067  0.7443902  0.41110906
 1.0000002 ]


 Checking distance of motor with the rest ['motors output exceeding 37', 'output exceeding 37', 'motors output', 'motors output exceeding', 'motors', 'exceeding 37', 'motor'] one by one
[0.55087155 0.51951694 0.8712373  0.72015566 0.94706273 0.5264343
 1.0000001 ]


 Checking distance of ac with the rest ['motors output exceeding 37', 'output exceeding 37', 'motors output', 'motors output exceeding', 'motors', 'exceeding 37', 'ac'] one by one
[0.4809123  0.56177974 0.6555984  0.58731705 0.7104611  0.5938185
 1.        ]


 max_sim  0.94706273





From Specification
('vehicles output 37 5w', 0.8172)
('motors vehicles output 37', 0.7849)

[0.4809123  0.56177974 0.6555984  0.58731705 0.7104611  0.5938185
 1.        ]


 max_sim  0.94706273





From Specification
('goods heading 84 15', 0.8434)
('goods heading 84', 0.8366)
('used goods heading 84', 0.8211)
('heading 84 15 84', 0.7579)
('84 15 84', 0.7457)
('84 18 84', 0.7294)
('heading 84', 0.7293)


From User Description
('use', 0.4857)
('ac motor industrial use', 0.9578)
('ac motor industrial', 0.918)
('ac motor', 0.8242)
('motor', 0.6969)
('ac', 0.6493)
('motor industrial', 0.818)


 Checking distance of use with the rest ['goods heading 84 15', 'goods heading 84', 'used goods heading 84', 'heading 84 15 84', '84 15 84', '84 18 84', 'use'] one by one
[0.48194772 0.4682594  0.44776988 0.53978705 0.5803634  0.6148958
 1.        ]


 Checking distance of ac motor industrial use with the rest ['goods heading 84 15', 'goods heading 84', 'used goods heading 84', 'heading 84 15 84', '84 15 84', '84 18 84', 'ac motor industrial use'] one by one
[0.38190556 0.35962698 0.370282

[0.8535117  0.78036296 0.87517405 0.79249424 0.6254864  0.8203716
 0.99999994]


 Checking distance of ac motor with the rest ['universal ac dc motors', 'universal ac dc', 'ac dc motors', 'dc motors', 'universal ac', 'ac dc', 'ac motor'] one by one
[0.87822783 0.82855254 0.93722457 0.8654847  0.6775107  0.8952814
 1.0000002 ]


 Checking distance of motor with the rest ['universal ac dc motors', 'universal ac dc', 'ac dc motors', 'dc motors', 'universal ac', 'ac dc', 'motor'] one by one
[0.60969985 0.582881   0.65923834 0.75814146 0.65156937 0.64417803
 1.0000001 ]


 Checking distance of ac with the rest ['universal ac dc motors', 'universal ac dc', 'ac dc motors', 'dc motors', 'universal ac', 'ac dc', 'ac'] one by one
[0.6490892  0.7668103  0.6655805  0.69274926 0.8359896  0.82472265
 1.        ]


 max_sim  0.93722457





From Specification
('wiper motor', 1.0)
('wiper', 0.9343)
('motor', 0.7036)


From User Description
('use', 0.4857)
('ac motor industrial use', 0.9578)
('ac motor

In [4]:
df.head(50)

,S.No,HSN_Code,Description,Unnamed: 3
0,1,8501.10.20,AC motor,Electric motors and generators (excluding gene...
1,4,8501.10.99,DC motors,Electric motors and generators (excluding gene...
2,5,8501.10.10,Fan motor fitted with revolving mechanism,Electric motors and generators (excluding gene...
3,6,8501.10.11,Micro motor,Electric motors and generators (excluding gene...
4,7,8501.10.00,Motors of an output not exceeding 37.5 ...,Electric motors and generators (excluding gene...
5,8,8501.10.00,Motors of an output not exceeding 37.5 W,Electric motors and generators (excluding gene...
6,9,8501.10.90.00,Motors For Other Than Vehicles Of An Output =<...,Electric motors and generators (excluding gene...
7,10,8501.10.10.00,Motors For Vehicles Of An Output =< 37.5W,Electric motors and generators (excluding gene...
8,11,8501.10.00,Motors of an output not,Electric motors and generators (excluding gene...
9,12,8501.10.00,Motors of an output not exceeding 37.5 W,Electric motors and generators (excluding gene...


In [5]:
# Works after preprocessing the text like bracket removal and all

def keep_unique(mystring):

    words = mystring.split()

    answer = [each_pair[0] for each_pair in zip(words, words[1:]) if each_pair[0] != each_pair[1]] + [words[-1]]

    print(' '.join(answer))

In [6]:
import nltk
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
from nltk.stem import WordNetLemmatizer

In [7]:
lemmatizer = WordNetLemmatizer()

In [8]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [9]:
def stem_lemmatize_text(text):
    text = text.lower()
    l = [stemmer.stem(w) for w in w_tokenizer.tokenize(text)]
    ls =  [lemmatizer.lemmatize(w) for w in l]
    listToStr = ' '.join([str(elem) for elem in ls])
    return listToStr

In [10]:
df['Des_lemmatized'] = df.Description.apply(stem_lemmatize_text)

In [11]:
df['Des_lemmatized'].head()

0                                ac motor
1                                dc motor
2        fan motor fit with revolv mechan
3                             micro motor
4    motor of an output not exceed 37.5 w
Name: Des_lemmatized, dtype: object

In [12]:
res  = []

In [13]:
def jaccard_sim(query):
    res = []
    for i in df['Des_lemmatized']:
        q = stem_lemmatize_text(query)
        li = i.split()
        lq = q.split()
        s1 = set(li)
        s2 = set(lq)
        res.append(float(len(s1.intersection(s2)) / 5))
    return res

In [14]:
jaccard_sim("ac fan motor revolv")

[0.4,
 0.2,
 0.6,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.0,
 0.0,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2]

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
import texthero as hero
df['tfidf'] = hero.do_tfidf(df['Des_lemmatized'])

In [17]:
df['tfidf'][0]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9283528855987047,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.37170004008684004,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

# idf lookup list

In [18]:
tf = TfidfVectorizer(use_idf=True)
tf.fit_transform(df['Des_lemmatized'])

idf = tf.idf_ 

In [19]:
print(idf)

[3.19722458 3.19722458 3.19722458 3.19722458 2.09861229 3.19722458
 2.79175947 3.19722458 3.19722458 2.79175947 1.69314718 2.79175947
 2.09861229 3.19722458 3.19722458 2.5040774  3.19722458 3.19722458
 3.19722458 3.19722458 3.19722458 1.11778304 1.94446161 1.58778666
 3.19722458 2.79175947 1.69314718 3.19722458 3.19722458 3.19722458
 3.19722458 3.19722458 3.19722458 3.19722458 3.19722458 2.79175947
 3.19722458 3.19722458]


In [20]:
tf.idf_[tf.vocabulary_["motor"]]

1.1177830356563834

# Create query idf Vector

In [21]:
def create_query_idf_vector(query):
    idf_v = []
    l = query.split()
    for q in l:
        idf_v.append(tf.idf_[tf.vocabulary_[q]])
    return idf_v

In [22]:
q = create_query_idf_vector("fan motor")

In [23]:
print(q)

[3.1972245773362196, 1.1177830356563834]


In [24]:
js = jaccard_sim("fan motor")

In [25]:
df.head()

,S.No,HSN_Code,Description,Unnamed: 3,Des_lemmatized,tfidf
0,1,8501.10.20,AC motor,Electric motors and generators (excluding gene...,ac motor,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,4,8501.10.99,DC motors,Electric motors and generators (excluding gene...,dc motor,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,5,8501.10.10,Fan motor fitted with revolving mechanism,Electric motors and generators (excluding gene...,fan motor fit with revolv mechan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,6,8501.10.11,Micro motor,Electric motors and generators (excluding gene...,micro motor,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,7,8501.10.00,Motors of an output not exceeding 37.5 ...,Electric motors and generators (excluding gene...,motor of an output not exceed 37.5 w,"[0.0, 0.0, 0.0, 0.0, 0.44648012504229373, 0.0,..."


In [26]:
print(js)

[0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.0, 0.0, 0.2, 0.2, 0.2, 0.2, 0.2]


In [27]:
def prod(qs,j):
    res = []
    for i in j:
        ls = [i*x for x in qs]
        res.append(ls)
    return res

In [28]:
res = prod(q,js)

In [29]:
res[0]

[0.6394449154672439, 0.22355660713127667]

In [30]:

import numpy as np
def mag(res):
    r = []
    for i in res:
        x = np.array(i)
        ans= np.linalg.norm(i)
        r.append(ans)
    return r
        


In [31]:
r = mag(res)

In [32]:
print(r)

[0.6773974878230349, 0.6773974878230349, 1.3547949756460698, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.0, 0.0, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349, 0.6773974878230349]


In [33]:
from re import sub

In [34]:
from gensim.utils import simple_preprocess

query_string = 'fruit and vegetables'
documents = ['cars drive on the road', 'tomatoes are actually fruit']

stopwords = ['the', 'and', 'are', 'a']


def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)


In [35]:
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

ImportError: cannot import name 'WordEmbeddingSimilarityIndex' from 'gensim.models' (D:\Anaconda\lib\site-packages\gensim\models\__init__.py)

In [ ]:
# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


In [ ]:
import gensim
from gensim import corpora
import math

class BM25 :
    def __init__(self, fn_docs, delimiter='|') :
        self.dictionary = corpora.Dictionary()
        self.DF = {}
        self.delimiter = delimiter
        self.DocTF = []
        self.DocIDF = {}
        self.N = 0
        self.DocAvgLen = 0
        self.fn_docs = fn_docs
        self.DocLen = []
        self.buildDictionary()
        self.TFIDF_Generator()

    def buildDictionary(self) :
        raw_data = []
        for line in file(self.fn_docs) :
            raw_data.append(line.strip().split(self.delimiter))
        self.dictionary.add_documents(raw_data)

    def TFIDF_Generator(self, base=math.e) :
        docTotalLen = 0
        for line in file(self.fn_docs) :
            doc = line.strip().split(self.delimiter)
            docTotalLen += len(doc)
            self.DocLen.append(len(doc))
            #print self.dictionary.doc2bow(doc)
            bow = dict([(term, freq*1.0/len(doc)) for term, freq in self.dictionary.doc2bow(doc)])
            for term, tf in bow.items() :
                if term not in self.DF :
                    self.DF[term] = 0
                self.DF[term] += 1
            self.DocTF.append(bow)
            self.N = self.N + 1
        for term in self.DF:
            self.DocIDF[term] = math.log((self.N - self.DF[term] +0.5) / (self.DF[term] + 0.5), base)
        self.DocAvgLen = docTotalLen / self.N

    def BM25Score(self, Query=[], k1=1.5, b=0.75) :
        query_bow = self.dictionary.doc2bow(Query)
        scores = []
        for idx, doc in enumerate(self.DocTF) :
            commonTerms = set(dict(query_bow).keys()) & set(doc.keys())
            tmp_score = []
            doc_terms_len = self.DocLen[idx]
            for term in commonTerms :
                upper = (doc[term] * (k1+1))
                below = ((doc[term]) + k1*(1 - b + b*doc_terms_len/self.DocAvgLen))
                tmp_score.append(self.DocIDF[term] * upper / below)
            scores.append(sum(tmp_score))
        return scores

    def TFIDF(self) :
        tfidf = []
        for doc in self.DocTF :
            doc_tfidf  = [(term, tf*self.DocIDF[term]) for term, tf in doc.items()]
            doc_tfidf.sort()
            tfidf.append(doc_tfidf)
        return tfidf

    def Items(self) :
        # Return a list [(term_idx, term_desc),]
        items = self.dictionary.items()
        items.sort()
        return items


In [52]:
import numpy as np

In [53]:
from rank_bm25 import BM25Okapi

corpus = [
    "ac motor",
    "dc motor",
    "fan motor fitted with revolving mechanism",
    "ac motor for household purpose",
    "ac for industrial use",
    "where is my order"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

#query = "ac motor having fan used in industrial purpose"
query = "ac motor"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)

In [54]:
type(doc_scores)

numpy.ndarray

In [55]:
print(doc_scores)

[0.34085801 0.34085801 0.21325778 0.23527675 0.         0.        ]


In [ ]:
from gensim.similarities import docsim
DocSim = docsim(verbose=True)
